In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import re

In [ ]:
def read_article(text):        
  sentences =[]        
  sentences = sent_tokenize(text)    
  for sentence in sentences:        
    sentence.replace("[^a-zA-Z0-9]"," ")     
  return sentences

In [ ]:
def sentence_similarity(sent1,sent2,stopwords=None):    
  if stopwords is None:        
    stopwords = []        
  sent1 = [w.lower() for w in sent1]    
  sent2 = [w.lower() for w in sent2]
        
  all_words = list(set(sent1 + sent2))   
     
  vector1 = [0] * len(all_words)    
  vector2 = [0] * len(all_words)        
  #build the vector for the first sentence    
  for w in sent1:        
    if not w in stopwords:
      vector1[all_words.index(w)]+=1                                                             
  #build the vector for the second sentence    
  for w in sent2:        
    if not w in stopwords:            
      vector2[all_words.index(w)]+=1 
               
  return 1-cosine_distance(vector1,vector2)

In [ ]:
def build_similarity_matrix(sentences,stop_words):
  #create an empty similarity matrix
  similarity_matrix = np.zeros((len(sentences),len(sentences)))
  for idx1 in range(len(sentences)):
      for idx2 in range(len(sentences)):
        if idx1!=idx2:
          similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2],stop_words)
  return similarity_matrix

In [ ]:
def generate_summary(text,top_n):
  nltk.download('stopwords')    
  nltk.download('punkt')
  stop_words = stopwords.words('english')    
  summarize_text = []
  # Step1: read text and tokenize    
  sentences = read_article(text)
  # Step2: generate similarity matrix            
  sentence_similarity_matrix = build_similarity_matrix(sentences,stop_words)
  # Step3: Rank sentences in similarity matrix
  sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
  scores = nx.pagerank(sentence_similarity_graph)
  # Step4: sort the rank and place top sentences
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
  
  # Step5: get the top n number of sentences based on rank
  for i in range(top_n):
    summarize_text.append(ranked_sentences[i][1])
  # Step6 : output the summarized version
  return " ".join(summarize_text),len(sentences)

In [ ]:
summary=generate_summary('''WASHINGTON - The Trump administration has ordered the military to start withdrawing roughly 7,000 troops from Afghanistan in the coming months, two defense officials said Thursday, an abrupt shift in the 17-year-old war there and a decision that stunned Afghan officials, who said they had not been briefed on the plans.
President Trump made the decision to pull the troops - about half the number the United States has in Afghanistan now - at the same time he decided to pull American forces out of Syria, one official said.
The announcement came hours after Jim Mattis, the secretary of defense, said that he would resign from his position at the end of February after disagreeing with the president over his approach to policy in the Middle East.
The whirlwind of troop withdrawals and the resignation of Mr. Mattis leave a murky picture for what is next in the United States’ longest war, and they come as Afghanistan has been troubled by spasms of violence afflicting the capital, Kabul, and other important areas. 
The United States has also been conducting talks with representatives of the Taliban, in what officials have described as discussions that could lead to formal talks to end the conflict.
Senior Afghan officials and Western diplomats in Kabul woke up to the shock of the news on Friday morning, and many of them braced for chaos ahead. 
Several Afghan officials, often in the loop on security planning and decision-making, said they had received no indication in recent days that the Americans would pull troops out. 
The fear that Mr. Trump might take impulsive actions, however, often loomed in the background of discussions with the United States, they said.
They saw the abrupt decision as a further sign that voices from the ground were lacking in the debate over the war and that with Mr. Mattis’s resignation, Afghanistan had lost one of the last influential voices in Washington who channeled the reality of the conflict into the White House’s deliberations.
The president long campaigned on bringing troops home, but in 2017, at the request of Mr. Mattis, he begrudgingly pledged an additional 4,000 troops to the Afghan campaign to try to hasten an end to the conflict.
Though Pentagon officials have said the influx of forces - coupled with a more aggressive air campaign - was helping the war effort, Afghan forces continued to take nearly unsustainable levels of casualties and lose ground to the Taliban.
The renewed American effort in 2017 was the first step in ensuring Afghan forces could become more independent without a set timeline for a withdrawal. 
But with plans to quickly reduce the number of American troops in the country, it is unclear if the Afghans can hold their own against an increasingly aggressive Taliban.
Currently, American airstrikes are at levels not seen since the height of the war, when tens of thousands of American troops were spread throughout the country. 
That air support, officials say, consists mostly of propping up Afghan troops while they try to hold territory from a resurgent Taliban.''',2)[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
summary

'The whirlwind of troop withdrawals and the resignation of Mr. Mattis leave a murky picture for what is next in the United States’ longest war, and they come as Afghanistan has been troubled by spasms of violence afflicting the capital, Kabul, and other important areas. They saw the abrupt decision as a further sign that voices from the ground were lacking in the debate over the war and that with Mr. Mattis’s resignation, Afghanistan had lost one of the last influential voices in Washington who channeled the reality of the conflict into the White House’s deliberations.'

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 182 kB 55.2 MB/s 
     |████████████████████████████████| 7.6 MB 43.4 MB/s 


In [ ]:
from transformers import pipeline
import os

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
text = """One month after the United States began what has become a troubled rollout of a national COVID vaccination campaign, the effort is finally gathering real steam.
Close to a million doses -- over 951,000, to be more exact -- made their way into the arms of Americans in the past 24 hours, the U.S. Centers for Disease Control and Prevention reported Wednesday. That's the largest number of shots given in one day since the rollout began and a big jump from the previous day, when just under 340,000 doses were given, CBS News reported.
That number is likely to jump quickly after the federal government on Tuesday gave states the OK to vaccinate anyone over 65 and said it would release all the doses of vaccine it has available for distribution. Meanwhile, a number of states have now opened mass vaccination sites in an effort to get larger numbers of people inoculated, CBS News reported."""

In [ ]:
summary_text = summarizer(text, max_length=100, min_length=5, do_sample=False)[0]['summary_text']
print(summary_text)

close to a million doses have been given in the past 24 hours, the cdc says . that's the largest number of shots given in one day since the rollout began . a number of states have opened mass vaccination sites to get more people inoculated .


In [ ]:
! wget https://archive.org/download/Insight-180206/Insight-180206d.mp3

--2022-11-17 09:02:27--  https://archive.org/download/Insight-180206/Insight-180206d.mp3
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia601401.us.archive.org/7/items/Insight-180206/Insight-180206d.mp3 [following]
--2022-11-17 09:02:27--  https://ia601401.us.archive.org/7/items/Insight-180206/Insight-180206d.mp3
Resolving ia601401.us.archive.org (ia601401.us.archive.org)... 207.241.227.121
Connecting to ia601401.us.archive.org (ia601401.us.archive.org)|207.241.227.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20867866 (20M) [audio/mpeg]
Saving to: ‘Insight-180206d.mp3’

Insight-180206d.mp3 100%[===================>]  19.90M  17.1MB/s    in 1.2s    

2022-11-17 09:02:29 (17.1 MB/s) - ‘Insight-180206d.mp3’ saved [20867866/20867866]



In [ ]:
!pip install git+https://github.com/openai/whisper.git -q
import whisper

In [ ]:
model = whisper.load_model("small")


100%|███████████████████████████████████████| 461M/461M [00:13<00:00, 36.3MiB/s]


In [ ]:
text = model.transcribe("/content/Insight-180206d.mp3")
#printing the transcribe
text['text']

/usr/local/lib/python3.7/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


" You're listening to Insight on Capitol Public Radio. I'm Beth Ruiak. If you're anywhere in the vicinity, happy bacon fest. Sacramento's Bacon Fest 7 kicks off tonight. There are events featuring bacon-themed dishes at all of the best restaurants across Sacramento. And it was even chosen on CNN as one of the world's best food festivals. How about that? So in the studio with me is Brian Guido. He started Bacon Fest 7 years ago, sort of trying to create a farm-to-fork celebration across restaurants in the city. Hi, Brian. Hey, how's it going? Glad to have you here. I'm going to call you Guido, okay? That's great. Which is good because I have Brad Chetshee here too. Hi, Brad. Hi, how are you? And Brad is executive chef and partner of Cannon East. Is there a restaurant without bacon anymore? I don't think so. Why would there be? All right, so for CNN to single out this festival says something, doesn't it? I was pretty much, I was taken aback by that. I didn't really expect that. And I gue

In [ ]:
! pip install gTTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
extractive_summary=generate_summary(text['text'],10)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
extractive_summary[0]

"I think the great thing about our events too is we focus on places that raise animals the right way and the restaurants that cure their own bacon and do their own charcuterie and use the whole animal as much as possible too. Brad, what do you... Well, I mean, there's a cocktail competition this week, which I think is pretty interesting to try to incorporate bacon flavors into booze. When I first had the idea and I went to talk to Patrick Mulvaney about it, and he's talked about a couple of events that were happening in Iowa and then Chicago. Tell me about Sacramento and the style of restaurant you have, which is neighborhood tavern, kind of American neighborhood food. So you'll get to choose your broth, the noodles and your protein. So people always ask us for a list of restaurants and honestly, I just can't do that because there's so many and I don't even know something that come up until that day. But for folks that want to participate somehow, but they're just not going the bacon r

In [ ]:
abstractive_summary=summarizer(text['text'], max_length=400, min_length=5, do_sample=False)[0]['summary_text']

In [ ]:
from gtts import gTTS #Import Google Text to Speech
from IPython.display import Audio #Import Audio method from IPython's Display Class
tts = gTTS(extractive_summary[0]) #Provide the string to convert to speech
tts.save('1.wav') #save the string converted to speech as a .wav file
sound_file = '1.wav'
Audio(sound_file, autoplay=True) 

In [ ]:
from gtts import gTTS #Import Google Text to Speech
from IPython.display import Audio #Import Audio method from IPython's Display Class
tts = gTTS(abstractive_summary) #Provide the string to convert to speech
tts.save('2.wav') #save the string converted to speech as a .wav file
sound_file = '2.wav'
Audio(sound_file, autoplay=True) 